In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import csv

In [15]:
transaction = pd.read_csv("transactions.csv", delimiter="|")
items = pd.read_csv("items.csv", delimiter="|")

In [17]:
transaction.head()

,sessionID,itemID,click,basket,order
0,0,21310,1,0,0
1,1,73018,1,0,0
2,2,19194,1,0,0
3,3,40250,1,0,0
4,4,46107,1,0,0


In [28]:
print(transaction.shape)
print(items.shape)

(365143, 5)
(78030, 6)


In [22]:
inner_merged_total = pd.merge(transaction, items, on=["itemID"])

In [24]:
inner_merged_total.shape

(365143, 10)